In [120]:
import pandas as pd
import numpy as np
import sklearn

(a)Download and select 70% of samples as training ser.

In [121]:
df = pd.read_csv("Frogs_MFCCs.csv")

In [122]:
df.shape

(7195, 26)

In [123]:
train_rows =  round(df.shape[0] * 0.7)
train_rows

5036

In [124]:
df = df.sample(frac=1).reset_index(drop=True)

In [125]:
df_train = df.iloc[:train_rows,:]
df_train.shape

(5036, 26)

In [126]:
df_test = df.iloc[train_rows:,]
df_test.shape

(2159, 26)

In [127]:
df_train.columns

Index(['MFCCs_ 1', 'MFCCs_ 2', 'MFCCs_ 3', 'MFCCs_ 4', 'MFCCs_ 5', 'MFCCs_ 6',
       'MFCCs_ 7', 'MFCCs_ 8', 'MFCCs_ 9', 'MFCCs_10', 'MFCCs_11', 'MFCCs_12',
       'MFCCs_13', 'MFCCs_14', 'MFCCs_15', 'MFCCs_16', 'MFCCs_17', 'MFCCs_18',
       'MFCCs_19', 'MFCCs_20', 'MFCCs_21', 'MFCCs_22', 'Family', 'Genus',
       'Species', 'RecordID'],
      dtype='object')

(b)i.Research exact match and hamming score/ loss methods.

Exact match is use all labels as metric, it's more easy to get a lower score.
Hamming score, on the contrary, measures each label respectively.

ii.Train a SVM for each of the labels, using Gaussian kernels and one versus all
classifiers.

In [128]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.model_selection import GridSearchCV

In [129]:
X, y1, y2, y3 = df_train.iloc[:,:22], df_train['Family'], df_train['Genus'], df_train['Species']

In [130]:
X.columns

Index(['MFCCs_ 1', 'MFCCs_ 2', 'MFCCs_ 3', 'MFCCs_ 4', 'MFCCs_ 5', 'MFCCs_ 6',
       'MFCCs_ 7', 'MFCCs_ 8', 'MFCCs_ 9', 'MFCCs_10', 'MFCCs_11', 'MFCCs_12',
       'MFCCs_13', 'MFCCs_14', 'MFCCs_15', 'MFCCs_16', 'MFCCs_17', 'MFCCs_18',
       'MFCCs_19', 'MFCCs_20', 'MFCCs_21', 'MFCCs_22'],
      dtype='object')

In [131]:
le = preprocessing.LabelEncoder()

In [132]:
le.fit(['Bufonidae','Dendrobatidae','Hylidae','Leptodactylidae'])
y1 = le.transform(y1) 

In [133]:
le.fit(['Adenomera','Ameerega','Dendropsophus','Hypsiboas','Leptodactylus','Osteocephalus',
        'Rhinella','Scinax'])
y2 = le.transform(y2) 

In [134]:
le.fit( ['AdenomeraAndre','AdenomeraHylaedactylus','Ameeregatrivittata','HylaMinuta',
         'HypsiboasCinerascens','HypsiboasCordobae','LeptodactylusFuscus',
         'OsteocephalusOophagus','Rhinellagranulosa','ScinaxRuber'])
y3 = le.transform(y3) 

In [155]:
X_test, y1_test, y2_test, y3_test = df_test.iloc[:,:22], df_test['Family'],\
                                    df_test['Genus'], df_test['Species']

In [156]:
le.fit(['Bufonidae','Dendrobatidae','Hylidae','Leptodactylidae'])
y1_test = le.transform(y1_test) 

In [158]:
le.fit(['Adenomera','Ameerega','Dendropsophus','Hypsiboas','Leptodactylus','Osteocephalus',
        'Rhinella','Scinax'])
y2_test = le.transform(y2_test) 

In [159]:
le.fit( ['AdenomeraAndre','AdenomeraHylaedactylus','Ameeregatrivittata','HylaMinuta',
         'HypsiboasCinerascens','HypsiboasCordobae','LeptodactylusFuscus',
         'OsteocephalusOophagus','Rhinellagranulosa','ScinaxRuber'])
y3_test = le.transform(y3_test) 

In [135]:
def standardize(df):
    df_std = df
    df_std = (df - df.mean()) / (df.max() - df.min() +0.00000000001)
    return df_std

In [160]:
X_std = standardize(X)
X_test_std = standardize(X_test)
X_std.iloc[0,:]

MFCCs_ 1    0.008595
MFCCs_ 2    0.001541
MFCCs_ 3   -0.150666
MFCCs_ 4    0.041563
MFCCs_ 5    0.097944
MFCCs_ 6   -0.025795
MFCCs_ 7   -0.054042
MFCCs_ 8    0.034341
MFCCs_ 9   -0.025505
MFCCs_10   -0.149778
MFCCs_11   -0.112897
MFCCs_12    0.087945
MFCCs_13    0.101505
MFCCs_14   -0.023486
MFCCs_15   -0.105090
MFCCs_16   -0.026013
MFCCs_17   -0.013274
MFCCs_18   -0.017516
MFCCs_19    0.020790
MFCCs_20   -0.021522
MFCCs_21    0.012474
MFCCs_22    0.037748
Name: 0, dtype: float64

In [168]:
parameters = {'C':[1, 10, 100], 'gamma':[1,2,3]}
svc = SVC(kernel='rbf')
cv = GridSearchCV(svc, parameters,cv=10)

In [153]:
def exact_match(y_true, y_pred):
    exa_score = 0
    for i in range(len(y_true[0])):
        if (y_true[0][i] == y_pred[0][i]) and (y_true[1][i] == y_pred[1][i]) \
                and (y_true[2][i] == y_pred[2][i]):
            exa_score += 1
            
    return exa_score/len(y_true[0])

First, for X without standardizing.

In [139]:
best_params = []
for labels in [y1,y2,y3]:
    cv.fit(X,labels)
    best_params.append(cv.best_params_)
    print(cv.best_params_)
    

{'C': 100, 'gamma': 2}


{'C': 100, 'gamma': 1}


{'C': 100, 'gamma': 2}


In [161]:
y_pred = []
labels = [y1,y2,y3]
for i in range(len(labels)):
    svc = SVC(kernel='rbf',C=best_params[i]['C'],gamma=best_params[i]['gamma'])
    ovr = OneVsRestClassifier(svc)
    ovr.fit(X,labels[i])
    y_pred.append(ovr.predict(X_test))

In [162]:
ham_loss = (hamming_loss(y1_test,y_pred[0]) + hamming_loss(y2_test, y_pred[1]) + 
            hamming_loss(y3_test,y_pred[2]))/3
print(ham_loss)

0.006638876022850085


In [163]:
exact_match([y1_test,y2_test,y3_test],y_pred)

0.9884205650764243

Second, with standardized X.

In [149]:
best_params_std = []
for labels in [y1,y2,y3]:
    cv.fit(X_std,labels)
    best_params_std.append(cv.best_params_)
    print(cv.best_params_)
    

{'C': 100, 'gamma': 3}


{'C': 100, 'gamma': 2}


{'C': 100, 'gamma': 2}


In [164]:
y_pred = []
labels = [y1,y2,y3]
for i in range(len(labels)):
    svc = SVC(kernel='rbf',C=best_params_std[i]['C'],gamma=best_params_std[i]['gamma'])
    ovr = OneVsRestClassifier(svc)
    ovr.fit(X_std,labels[i])
    y_pred.append(ovr.predict(X_test_std))

In [165]:
ham_loss = (hamming_loss(y1_test,y_pred[0]) + hamming_loss(y2_test, y_pred[1]) + 
            hamming_loss(y3_test,y_pred[2]))/3
print(ham_loss)

0.015748031496062992


In [166]:
exact_match([y1_test,y2_test,y3_test],y_pred)

0.9749884205650764

By result, data without standardizing will give a better prediction.

(b).iii.Repeat 1(b)ii with L1-penalized SVMs. Remember to normalize the attributes.

In [167]:
from sklearn.svm import LinearSVC

In [177]:
parameters = {'C':[1, 10, 100]}
svc = LinearSVC(penalty='l1',dual=False)
cv = GridSearchCV(svc, parameters,cv=10)

In [178]:
best_params = []
for labels in [y1,y2,y3]:
    cv.fit(X,labels)
    best_params.append(cv.best_params_)
    print(cv.best_params_)
    

{'C': 1}


{'C': 100}


{'C': 100}


In [185]:
y_pred = []
labels = [y1,y2,y3]
for i in range(len(labels)):
    svc = LinearSVC(penalty='l1',C=best_params[i]['C'],dual=False)
    ovr = OneVsRestClassifier(svc)
    ovr.fit(X,labels[i])
    y_pred.append(ovr.predict(X_test))

In [186]:
ham_loss = (hamming_loss(y1_test,y_pred[0]) + hamming_loss(y2_test, y_pred[1]) + 
            hamming_loss(y3_test,y_pred[2]))/3
print(ham_loss)

0.05002315886984715


In [187]:
exact_match([y1_test,y2_test,y3_test],y_pred)

0.9170912459471978

iv. Repeat 1(b)iii by using SMOTE

In [189]:
from imblearn.over_sampling import SMOTE 

In [191]:
sm = SMOTE()

In [192]:
X1_res, y1_res = sm.fit_sample(X, y1)
X2_res, y2_res = sm.fit_sample(X, y2)
X3_res, y3_res = sm.fit_sample(X, y3)
X1_test_res, y1_test_res = sm.fit_sample(X_test, y1_test)
X2_test_res, y2_test_res = sm.fit_sample(X_test, y2_test)
X3_test_res, y3_test_res = sm.fit_sample(X_test, y3_test)

In [190]:
parameters = {'C':[1, 10, 100]}
svc = LinearSVC(penalty='l1',dual=False)
cv = GridSearchCV(svc, parameters,cv=10)

In [193]:
best_params = []
data_sets = [X1_res,X2_res,X3_res]
labels = [y1_res,y2_res,y3_res]
for i in range(len(labels)):
    cv.fit(data_sets[i],labels[i])
    best_params.append(cv.best_params_)
    print(cv.best_params_)
    

{'C': 10}


KeyboardInterrupt: 

In [ ]:
y_pred = []
test_data_sets = [X1_test_res,X2_test_res,X3_test_res]
for i in range(len(labels)):
    svc = LinearSVC(penalty='l1',C=best_params[i]['C'],dual=False)
    ovr = OneVsRestClassifier(svc)
    ovr.fit(data_sets[i],labels[i])
    y_pred.append(ovr.predict(test_data_sets[i]))

In [186]:
ham_loss = (hamming_loss(y1_test_res,y_pred[0]) + hamming_loss(y2_test_res, y_pred[1]) + 
            hamming_loss(y3_test_res,y_pred[2]))/3
print(ham_loss)

0.05002315886984715


In [187]:
exact_match([y1_test_res,y2_test_res,y3_test_res],y_pred)

0.9170912459471978